In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tnrange
from urllib.request import urlopen
import re
import requests
import urllib.request
from tqdm import tqdm
from konlpy.tag import Okt
from selenium.common.exceptions import NoSuchElementException 

In [2]:
train = pd.read_csv("./data/train.csv")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_options=options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [4]:
def preprocess_nlp(dataset: pd.DataFrame, train: bool) -> pd.DataFrame:
    f = open("./data/stopword.txt", encoding="UTF-8")
    line = f.readlines()
    stopwords = []

    for l in line:
        l = l.replace("\n", "")
        stopwords.append(l)

    print("### Start Preprocess for Overview ###")
    drop_list = []
    for idx in tqdm(range(len(dataset))):
        to_fix = dataset["overview"][idx]
        to_fix = re.sub("<.+?>", "", to_fix)
        to_fix = re.sub("[^ 가-힣0-9a-zA-Z]", "", to_fix)

        # # Stopword
        # okt = Okt()
        # to_fix_morphs = okt.morphs(to_fix)
        # fixed_list = [w for w in to_fix_morphs if w not in stopwords]
        # to_fix_finish = " ".join(fixed_list)
        # if train and len(to_fix_finish) == 0:
        #     drop_list.append(idx)
        # if not train and len(to_fix_finish) == 0:
        #     to_fix_finish = to_fix
        # dataset["overview"][idx] = to_fix_finish

    if train:
        dataset.drop(drop_list)
    return dataset

In [5]:
def trans(t, driver):
    new_text = []
    lang = ['en']
    for l in lang:
        backtrans = ""
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+l+'&st='+t)
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
        except:
            pass
            # driver.get('https://papago.naver.com/?sk=ko&tk='+l)
            # driver.find_element_by_xpath('//*[@id="txtSource"]').send_keys(t)
            # time.sleep(2.5)
            # backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
        return_sen = ""
        if backtrans != "":
            try:
                driver.get('https://papago.naver.com/?sk='+l+'&tk=ko&st='+backtrans)
                time.sleep(2.5)
                return_sen = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            except:
                pass
        if return_sen != "":
            new_text.append(return_sen)
    return new_text

In [6]:
def backtranslation_aug(driver):
    train = pd.read_csv("./data/train.csv")
    train = preprocess_nlp(train, train=True)

    # aug_cat extract
    aug_cat = []
    val = train["cat3"].values.tolist()
    val = list(set(val))

    for v in val:
        if len(train[train["cat3"] == v]) < 200:
            aug_cat.append(v)

    for a in tqdm(aug_cat):
        aug = train[train["cat3"] == a]
        for idx in tqdm(range(len(aug))):
            origin = aug.iloc[idx].copy()
            aug_view = aug["overview"].iloc[idx]
            aug_view = re.sub("<br>", "", aug_view)
            # print(aug_view)
            aug_list = trans(aug_view, driver)
            aug_list = list(set(aug_list))

            for au in aug_list:
                # print(au)
                new_data = origin.copy()
                new_data["overview"] = au
                train = train.append(new_data)
    train.to_csv("./data/train_aug.csv", index=False)    

In [7]:
backtranslation_aug(driver)

### Start Preprocess for Overview ###


100%|██████████| 105/105 [6:41:46<00:00, 229.59s/it]
